In [21]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "/content/sample_data/oil_data.csv"  # Adjust the path as needed
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())


         Date  Price   Open   High    Low     Vol. Change %
0  12/06/2024  67.20  68.44  68.49  66.98      NaN   -1.61%
1  12/05/2024  68.30  68.74  69.16  67.98  286.70K   -0.35%
2  12/04/2024  68.54  70.02  70.51  68.49  325.64K   -2.00%
3  12/03/2024  69.94  68.16  70.23  67.91  303.75K    2.70%
4  12/02/2024  68.10  68.00  69.11  67.71  264.88K    0.15%


In [22]:
# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])
# Display the updated DataFrame (optional)
print(data.head())


        Date  Price   Open   High    Low     Vol. Change %
0 2024-12-06  67.20  68.44  68.49  66.98      NaN   -1.61%
1 2024-12-05  68.30  68.74  69.16  67.98  286.70K   -0.35%
2 2024-12-04  68.54  70.02  70.51  68.49  325.64K   -2.00%
3 2024-12-03  69.94  68.16  70.23  67.91  303.75K    2.70%
4 2024-12-02  68.10  68.00  69.11  67.71  264.88K    0.15%


In [23]:
# Sort by date in ascending order
data = data.sort_values(by='Date').reset_index(drop=True)
print(data.head())

        Date  Price   Open   High    Low     Vol. Change %
0 2014-01-01  98.70  98.61  98.76  98.54      NaN    0.28%
1 2014-01-02  95.44  98.50  98.97  95.34  251.25K   -3.30%
2 2014-01-03  93.96  95.47  95.74  93.86  215.52K   -1.55%
3 2014-01-06  93.43  94.18  94.59  93.20  189.38K   -0.56%
4 2014-01-07  93.67  93.60  94.22  93.35  177.18K    0.26%


In [24]:
# Function to convert volume strings to numeric values
def convert_volume(vol):
    if isinstance(vol, str):
        if 'K' in vol:
            return float(vol.replace('K', '')) * 1e3
        elif 'M' in vol:
            return float(vol.replace('M', '')) * 1e6
    return float(vol)  # Handle cases where no suffix is present

# Apply the conversion function to the 'Vol.' column
data['Volume'] = data['Vol.'].apply(convert_volume)

# Drop the original 'Vol.' column if no longer needed
data = data.drop(columns=['Vol.'])

# Check the result
print(data[['Date', 'Volume']].head())


        Date    Volume
0 2014-01-01       NaN
1 2014-01-02  251250.0
2 2014-01-03  215520.0
3 2014-01-06  189380.0
4 2014-01-07  177180.0


In [25]:

# Compute daily log returns
data['Log_Returns'] = np.log(data['Price'] / data['Price'].shift(1))

# Drop the first row with NaN from the log return calculation
data = data.dropna(subset=['Log_Returns']).reset_index(drop=True)


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [26]:
print(data.head())

        Date  Price   Open   High    Low Change %    Volume  Log_Returns
0 2014-01-02  95.44  98.50  98.97  95.34   -3.30%  251250.0    -0.033587
1 2014-01-03  93.96  95.47  95.74  93.86   -1.55%  215520.0    -0.015629
2 2014-01-06  93.43  94.18  94.59  93.20   -0.56%  189380.0    -0.005657
3 2014-01-07  93.67  93.60  94.22  93.35    0.26%  177180.0     0.002565
4 2014-01-08  92.33  93.96  94.18  92.26   -1.43%  244860.0    -0.014409


In [27]:
# Compute 7-day realized volatility
data['Realized_Volatility'] = data['Log_Returns'].rolling(window=7).std()

# Drop rows with NaN values from rolling calculation
data = data.dropna(subset=['Realized_Volatility']).reset_index(drop=True)

print(data.head())

        Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0 2014-01-10  92.72  92.30  93.38  91.99    1.16%  255990.0     0.011498   
1 2014-01-13  91.80  92.83  92.88  91.43   -0.99%  212210.0    -0.009972   
2 2014-01-14  92.59  91.51  92.88  91.50    0.86%  243470.0     0.008569   
3 2014-01-15  94.17  92.65  94.64  92.43    1.71%  263300.0     0.016921   
4 2014-01-16  93.96  94.29  94.64  93.60   -0.22%  165140.0    -0.002232   

   Realized_Volatility  
0             0.014409  
1             0.009653  
2             0.009777  
3             0.011900  
4             0.011942  


In [28]:
# 7-day moving average of Price
data['Moving_Avg_7'] = data['Price'].rolling(window=7).mean()

print(data.head())

        Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0 2014-01-10  92.72  92.30  93.38  91.99    1.16%  255990.0     0.011498   
1 2014-01-13  91.80  92.83  92.88  91.43   -0.99%  212210.0    -0.009972   
2 2014-01-14  92.59  91.51  92.88  91.50    0.86%  243470.0     0.008569   
3 2014-01-15  94.17  92.65  94.64  92.43    1.71%  263300.0     0.016921   
4 2014-01-16  93.96  94.29  94.64  93.60   -0.22%  165140.0    -0.002232   

   Realized_Volatility  Moving_Avg_7  
0             0.014409           NaN  
1             0.009653           NaN  
2             0.009777           NaN  
3             0.011900           NaN  
4             0.011942           NaN  


In [29]:
def compute_rsi(data, window=14):
    delta = data['Price'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Add RSI to the dataset (14-day window)
data['RSI_14'] = compute_rsi(data, window=14)

print(data)


           Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0    2014-01-10  92.72  92.30  93.38  91.99    1.16%  255990.0     0.011498   
1    2014-01-13  91.80  92.83  92.88  91.43   -0.99%  212210.0    -0.009972   
2    2014-01-14  92.59  91.51  92.88  91.50    0.86%  243470.0     0.008569   
3    2014-01-15  94.17  92.65  94.64  92.43    1.71%  263300.0     0.016921   
4    2014-01-16  93.96  94.29  94.64  93.60   -0.22%  165140.0    -0.002232   
...         ...    ...    ...    ...    ...      ...       ...          ...   
2865 2024-12-02  68.10  68.00  69.11  67.71    0.15%  264880.0     0.001470   
2866 2024-12-03  69.94  68.16  70.23  67.91    2.70%  303750.0     0.026661   
2867 2024-12-04  68.54  70.02  70.51  68.49   -2.00%  325640.0    -0.020220   
2868 2024-12-05  68.30  68.74  69.16  67.98   -0.35%  286700.0    -0.003508   
2869 2024-12-06  67.20  68.44  68.49  66.98   -1.61%       NaN    -0.016237   

      Realized_Volatility  Moving_Avg_7     RSI_14 

In [30]:
from sklearn.preprocessing import MinMaxScaler

# Select features to scale
features_to_scale = ['Log_Returns', 'Realized_Volatility', 'Moving_Avg_7', 'RSI_14']
features_to_scale.append('Volume')

scaler = MinMaxScaler()

# Scale the selected features
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

print(data.head())

        Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0 2014-01-10  92.72  92.30  93.38  91.99    1.16%  0.136322     0.488011   
1 2014-01-13  91.80  92.83  92.88  91.43   -0.99%  0.111348     0.452337   
2 2014-01-14  92.59  91.51  92.88  91.50    0.86%  0.129180     0.483143   
3 2014-01-15  94.17  92.65  94.64  92.43    1.71%  0.140492     0.497020   
4 2014-01-16  93.96  94.29  94.64  93.60   -0.22%  0.084496     0.465196   

   Realized_Volatility  Moving_Avg_7  RSI_14  
0             0.058984           NaN     NaN  
1             0.035325           NaN     NaN  
2             0.035938           NaN     NaN  
3             0.046504           NaN     NaN  
4             0.046710           NaN     NaN  


In [31]:
# Drop rows with NaN values caused by rolling calculations
data = data.dropna(subset=['RSI_14', 'Moving_Avg_7']).reset_index(drop=True)

# Check the resulting dataset
print(data.head())


        Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0 2014-01-29  97.36  97.17  97.75  96.32   -0.05%  0.104662     0.468053   
1 2014-01-30  98.23  97.42  98.59  97.33    0.89%  0.104051     0.483687   
2 2014-01-31  97.49  97.97  98.39  97.10   -0.75%  0.134879     0.456341   
3 2014-02-03  96.43  97.40  97.94  96.26   -1.09%  0.157612     0.450741   
4 2014-02-04  97.19  96.53  97.82  96.37    0.79%  0.106824     0.481950   

   Realized_Volatility  Moving_Avg_7    RSI_14  
0             0.043139      0.774725  0.748484  
1             0.042255      0.779125  0.772899  
2             0.037536      0.780157  0.785792  
3             0.040689      0.778949  0.690880  
4             0.041454      0.779696  0.662161  


In [32]:
# Confirm no missing values remain in critical columns
print(data.isnull().sum())

Date                     0
Price                    0
Open                     0
High                     0
Low                      0
Change %                 0
Volume                 125
Log_Returns              0
Realized_Volatility      0
Moving_Avg_7             0
RSI_14                   0
dtype: int64


In [34]:
# Drop rows where Volume is NaN
data = data.dropna(subset=['Volume']).reset_index(drop=True)
# Check for missing values in the dataset
print(data.isnull().sum())

# Verify the size of the dataset after dropping rows
print(f"Dataset size after dropping rows: {len(data)} rows")

Date                   0
Price                  0
Open                   0
High                   0
Low                    0
Change %               0
Volume                 0
Log_Returns            0
Realized_Volatility    0
Moving_Avg_7           0
RSI_14                 0
dtype: int64
Dataset size after dropping rows: 2732 rows


In [35]:
#f rows are dropped, the indices may no longer reflect continuous dates. To maintain continuity:
data = data.sort_values(by='Date').reset_index(drop=True)

In [36]:
print(data.head())

        Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0 2014-01-29  97.36  97.17  97.75  96.32   -0.05%  0.104662     0.468053   
1 2014-01-30  98.23  97.42  98.59  97.33    0.89%  0.104051     0.483687   
2 2014-01-31  97.49  97.97  98.39  97.10   -0.75%  0.134879     0.456341   
3 2014-02-03  96.43  97.40  97.94  96.26   -1.09%  0.157612     0.450741   
4 2014-02-04  97.19  96.53  97.82  96.37    0.79%  0.106824     0.481950   

   Realized_Volatility  Moving_Avg_7    RSI_14  
0             0.043139      0.774725  0.748484  
1             0.042255      0.779125  0.772899  
2             0.037536      0.780157  0.785792  
3             0.040689      0.778949  0.690880  
4             0.041454      0.779696  0.662161  


In [37]:
# Export the updated DataFrame to a CSV file
output_file_path = "/content/processed_oil_data.csv"  # Update path as needed
data.to_csv(output_file_path, index=False)

print(f"DataFrame exported to {output_file_path}")


DataFrame exported to /content/processed_oil_data.csv


In [38]:
# Verify the exported file
imported_data = pd.read_csv(output_file_path)
print(imported_data.head())


         Date  Price   Open   High    Low Change %    Volume  Log_Returns  \
0  2014-01-29  97.36  97.17  97.75  96.32   -0.05%  0.104662     0.468053   
1  2014-01-30  98.23  97.42  98.59  97.33    0.89%  0.104051     0.483687   
2  2014-01-31  97.49  97.97  98.39  97.10   -0.75%  0.134879     0.456341   
3  2014-02-03  96.43  97.40  97.94  96.26   -1.09%  0.157612     0.450741   
4  2014-02-04  97.19  96.53  97.82  96.37    0.79%  0.106824     0.481950   

   Realized_Volatility  Moving_Avg_7    RSI_14  
0             0.043139      0.774725  0.748484  
1             0.042255      0.779125  0.772899  
2             0.037536      0.780157  0.785792  
3             0.040689      0.778949  0.690880  
4             0.041454      0.779696  0.662161  
